# Controlling Cameras
In this example we will walk through how to control the main camera, take screenshots, and demonstrate the multiple camera functionality.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/VirtualBrainLab/urchin-examples/blob/main/basics/multi_camera.ipynb)

## Install Urchin

Urchin is a Python package stored on PyPI, the following code needs to be run the first time you use Urchin in a Python environment. 

Urchin's full documentation can be found [on our website](https://virtualbrainlab.org/urchin/installation_and_use.html).

In [ ]:
#Installing urchin
!pip install oursin -U

## Setup Urchin and open the renderer webpage 

By default Urchin opens the 3D renderer in a webpage. Make sure pop-ups are enabled, or the page won't open properly. You can also open the renderer site yourself by replacing [ID here] with the ID that is output by the call to `.setup()` at https://data.virtualbrainlab.org/Urchin/?ID=[ID here]

Note that Urchin communicates to the renderer webpage through an internet connection, we don't currently support offline use (we hope to add support in the future).

In [ ]:
#Importing necessary libraries:
import pandas as pd
import oursin as urchin
urchin.setup()

This tutorial uses data from a collected dataset, so the following data needs to be pulled in properly to display.

In [ ]:
#Function for easily pulling in data:
#url = link to google sheet (on editing mode) as a string
def get_data(url):
    data = url.replace("/edit#gid=", "/export?format=csv&gid=")
    df = pd.read_csv(data)
    return df

In [ ]:
# To pull in data to a dataframe:
data = get_data('https://docs.google.com/spreadsheets/d/1F9NBt-qqcA-IyxowXl82S4NI0gYyczUOEb8MEaW7qm0/edit#gid=1956783922')

## Displaying overall brain + corresponding brain regions
The following blocks enables the proper brain view, to give the camera a subject of focus. See the brain areas tutorial for more information.

In [ ]:
#Generating overall brain image:
urchin.ccf.set_visibility({'grey':True})
urchin.ccf.set_material({'grey':'transparent-unlit'})
urchin.ccf.set_alpha({'grey':0.1})

In [ ]:
#Establishing the brain regions we'll be using specifically
brain_areas = ["MRN", "SCs", "SCm", "APN", "RN"]
#Generating brain area images:
for area in brain_areas:
    urchin.ccf.set_visibility({area:True})
    urchin.ccf.set_material({area:'transparent-unlit'})
    urchin.ccf.set_alpha({area:0.2})

## Displaying neuron data in default display
Displaying neurons from the dataset. See the neurons and probes tutorial for more information.

In [ ]:
#Creating neurons
num_neurons = len(data)
neurons = urchin.neurons.create(num_neurons)

#Setting neuron colors
urchin.neurons.set_colors(neurons,list(data.loc[:,"color"]))

#Setting neuron positions
#putting in proper formating:
neuron_positions = []
for neuron in range(len(data)):
    neuron_pos = [data.loc[neuron, "x"], data.loc[neuron, "y"], data.loc[neuron, "z"]]
    neuron_positions.append(neuron_pos)
#Setting position:
urchin.neurons.set_positions(neurons, neuron_positions)

## Displaying additional views
The 'main' camera is the built in camera within urchin. It can be accessed directly via urchin.camera.main, but it is easier to store it as a python variable when handling multiple cameras at once.

In [ ]:
#Grabbing main camera as a python variable
main = urchin.camera.main

The following code instantiates a camera object within Urchin, and the names are stored as python variables to easily change their settings later.

In [ ]:
#Creating cameras c1, c2 and c3
c1 = urchin.camera.Camera()
c2 = urchin.camera.Camera()
c3 = urchin.camera.Camera()

In [ ]:
#Deleting camera c2
c2.delete()

### Changing rotation of cameras
The cameras are fixed around a central point, so changing the rotation changes the view of the camera.
Rotation coordinates are in format [pitch, yaw, spin] (As of v 0.4.7, the pitch and yaw are flipped on the rotation display in the corner).

In [ ]:
c1.set_rotation([90,0,0])

In [ ]:
c3.set_rotation([90,-90,0])

### Changing zoom or pan

Zoom is passed in as a float, pan coordinates are in format (x coordinate, y coordinate).

In [ ]:
c1.set_zoom(2.5)

In [ ]:
c3.set_pan(3, 0)

In [ ]:
c3.set_zoom(3)

### Targetting specific areas
This focuses the camera on one specific area, and can be done by either using target brain regions or target coordinates.

Highlighted brain areas: "MRN", "SCs", "SCm", "APN", "RN"

In [ ]:
#highlighted brain areas currently don't work

In [ ]:
c1.set_target_area("grey-lh")

In [ ]:
c3.set_target_coordinate([3850, 8700.0, 3200])

## Changing Camera settings

### Changing which camera is mouse controllable
To control the secondary cameras with the mouse by clicking and dragging, you can call `Camera.set_controllable()`.

In [ ]:
c1.set_controllable()

In [ ]:
main.set_controllable()

### Changing camera mode
There are two different camera modes used within Urchin: perspective, and orthographic. Use the `camera.set_mode()` function to change between the two modes.

In [ ]:
c1.set_mode('perspective')

In [ ]:
c1.set_mode('orthographic')

## Taking screenshots
You can take screenshots from any camera, and have them saved locally. Be sure to select the proper camera, and name the screenshot something memorable.

In [ ]:
c1.screenshot('./c1_screenshot.png')

To download files from the Colab UI, click the file icon on the left bar. Double click to view, right click to download.